# Advanced Univariate Modelling

In [ ]:
#r "nuget: Deedle, 2.3.0"
#r "nuget: Plotly.NET, 2.0.0-beta9"
#r "nuget: Plotly.NET.Interactive, 2.0.0-beta9"
#r "nuget: Microsoft.ML, 1.5.5"
#r "nuget: Microsoft.ML.TimeSeries, 1.5.5"
#r "nuget: FSharp.Stats, 0.4.1"

#i "nuget:https://www.myget.org/F/gregs-experimental-packages/api/v3/index.json"
#r "nuget:Deedle.DotNet.Interactive.Extension, 0.1.0-alpha6"

In [ ]:
open Deedle
open Plotly.NET

let data =
    Frame.ReadCsv("../data/at_load_hourly_mw.csv", hasHeaders = true, culture = "en-US", inferTypes = true, inferRows = 5_000)
    |> Frame.dropCol "Ticks"
    |> Frame.indexRowsDate "TimeStamp"

let dataTrain =
    data
    |> Frame.filterRows (fun key _ -> key.Year < 2019)

let dataTest =
    data
    |> Frame.filterRows (fun key _ -> key.Year >= 2019)

dataTrain
|> Frame.skip 3

In [ ]:
open Microsoft.ML
open Microsoft.ML.Data
open Microsoft.ML.Trainers
open Microsoft.ML.Transforms
open FSharp.Stats.Correlation
open Plotly.NET

[<CLIMutable>]
type AlternativeForecastInput =
    { Load: float32
      TimeStamp: DateTime }

[<CLIMutable>]
type AlternativeLoadForecast =
  { Forecast: float32 array
    LowerBound: float32 array
    UpperBound: float32 array }

let altForecastInputs =
  dataTrain?Value
  |> Series.observations
  |> Seq.map (fun (k, v) -> { Load = float32 v; TimeStamp = k})

let testKeys, testRows =
    dataTest?Value
    |> Series.observations
    |> Seq.unzip

In [ ]:
let mlContext = MLContext(seed = 42)

let pipeline =
  mlContext.Forecasting.ForecastBySsa(
    "Forecast",
    nameof Unchecked.defaultof<AlternativeForecastInput>.Load,
    windowSize =  24 * 30,
    seriesLength = 24 * 30 * 2,
    trainSize = dataTrain.RowCount,
    horizon = 24 * 30 ,
    confidenceLevel = 0.90f,
    confidenceLowerBoundColumn = "LowerBound",
    confidenceUpperBoundColumn = "UpperBound"
)

let altForecastData = mlContext.Data.LoadFromEnumerable(altForecastInputs)

let model = pipeline.Fit(altForecastData)

In [ ]:
open Microsoft.ML.Transforms.TimeSeries

let forecastingEngine = model.CreateTimeSeriesEngine<AlternativeForecastInput, AlternativeLoadForecast>(mlContext)

let horizon = 24 * 5
let forecast = forecastingEngine.Predict(horizon = horizon)

In [ ]:
let predChart =
    Seq.zip (testKeys |> Seq.take horizon) forecast.Forecast
    |> fun xy -> Chart.Range(xy,
                             forecast.LowerBound,
                             forecast.UpperBound,
                             mode = StyleParam.Mode.Lines,
                             Color = Colors.toWebColor Colors.Table.Office.blue,
                             RangeColor = Colors.toWebColor Colors.Table.Office.lightBlue)
    |> Chart.withTraceName "Forecast_CI"

let actualChart =
    Seq.zip (testKeys |> Seq.take horizon) (testRows |> Seq.take horizon)
    |> fun xy -> Chart.Line(xy,
                            Color = Colors.toWebColor Colors.Table.Office.orange,
                            UseWebGL = true,
                            Name = "Actual")

[ actualChart; predChart ]
|> Chart.Combine

In [ ]:
let actualVals = testRows |> Seq.take (24 * 30) |> Seq.map float32
let predVals = forecast.Forecast

let minVal =
    min (Seq.min predVals) (Seq.min actualVals)
    |> float
    |> fun v -> v - 100.

let largestVal =
    max (Seq.max predVals) (Seq.max actualVals)
    |> float
    |> fun v -> v + 100.

let diagonalLine =
    [ (minVal, minVal); (largestVal, largestVal) ]
    |> fun xy -> Chart.Line(xy, Name = "Diagonal")

let predActualScatter =
    Seq.zip predVals actualVals
    |> fun xy -> Chart.Point(xy, UseWebGL = true, Name = "Pred/Actual")
    |> Chart.withX_AxisStyle ("predictions", MinMax = (minVal, largestVal))
    |> Chart.withY_AxisStyle ("actual", MinMax = (minVal, largestVal))

[ predActualScatter; diagonalLine ]
|> Chart.Combine
|> display

Seq.pearson actualVals predVals
|> display

In [ ]:
let modelDirectory = "../models"
let forecastModel = modelDirectory + "/forecast_model.zip"

mlContext.Model.Save(model, altForecastData.Schema, forecastModel)